# $\nabla v_{LSR}$ in HC$_3$N

We calculate the gradient in velocities $v_{LSR}$ adn then visualize it with line integral convolution (LIC). 

For this, we calculate the gradient in a plane around a radius of approx 2 beam sizes, to sample well resolved structures.

In [1]:
import numpy as np
import scipy.linalg # we only fit a linear plane so we go with easy linear code
from astropy.io import fits
import astropy.units as u

In [2]:
gaussfitfolder = 'gaussfit/'
velfile = gaussfitfolder + 'B5-NOEMA+30m-H3CN-10-9_cut_K_1G_fitparams_filtered_Vlsr'
nablavelfile = velfile + '_gradient'

In [11]:
# test: data that has to be given as input

mean = np.array([0.0,0.0,0.0])
cov = np.array([[1.0,-0.5,0.8], [-0.5,1.1,0.0], [0.8,0.0,1.0]])
data = np.random.multivariate_normal(mean, cov, 50) # is (number of points, [x, y, z])

# regular grid covering the domain of the data
X,Y = np.meshgrid(np.arange(-3.0, 3.0, 0.5), np.arange(-3.0, 3.0, 0.5))
XX = X.flatten()
YY = Y.flatten()

A = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])] # Design matrix, where we have X, Y and a constant value (we just use 1)
C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])    # coefficients, magnitude in X, magnitude in Y and constant

In [13]:
C

array([ 0.98033152,  0.42068716, -0.05339694])